In [1]:
# importing the necessary libraries and helper scripts

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os
import logging
import argparse
import random
from tqdm import tqdm, trange

import yaml
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from helper.tokenization import BertTokenizer
from helper.modeling import BertForSequenceClassification
from helper.optimization import BertAdam
from helper.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
# loading the config files with yaml

with open("./config/global_config.yml", "r") as filename:
    config = yaml.load(filename)
data_dir = config["train_bert"]["data_dir"]
bert_model= config["train_bert"]["bert_model"]
task_name = config["train_bert"]["task_name"]
output_dir = config["train_bert"]["output_dir"]
with open(config["train_bert"]["labels"], 'r') as read_file:
    labels = list(map(lambda x: x.strip('\n'), read_file.readlines()))

# other parameters
do_train = config["train_bert"]["do_train"]
do_eval = config["train_bert"]["do_eval"]
max_seq_length = config["train_bert"]["max_seq_length"]
do_lower_case = config["train_bert"]["do_lower_case"]
train_batch_size = config["train_bert"]["train_batch_size"]
eval_batch_size = config["train_bert"]["eval_batch_size"]
learning_rate = config["train_bert"]["learning_rate"]
num_train_epochs = config["train_bert"]["num_train_epochs"]
learning_rate = float(config["train_bert"]["learning_rate"]) 
warmup_proportion = config["train_bert"]["warmup_proportion"]
no_cuda = config["train_bert"]["no_cuda"]
local_rank = config["train_bert"]["local_rank"]
seed = config["train_bert"]["seed"]
gradient_accumulation_steps = config["train_bert"]["gradient_accumulation_steps"]
fp16 = config["train_bert"]["fp16"]
loss_scale = config["train_bert"]["loss_scale"]

In [3]:
# Reading Data and preprocessing

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()
    
    def set_labels(self, labels):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding='utf-8') as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines

class ToxicCommentsProcessor(DataProcessor):
    """Processor for the Toxic Comments data set ."""
    def __init__(self):
        self.labels = None

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")
    
    def set_labels(self, labels):
        self.labels = labels
        
    def get_labels(self):
        """See base class."""
        return self.labels

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            try:
                guid = "%s-%s" % (set_type, i)
                if line[0] and line[1]:
                        text_a = line[0]
                        label = line[1]
                        examples.append(
                            InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
            except:
                pass
        return examples
    
    def convert_examples_to_features(self, examples, label_list, max_seq_length, tokenizer):
        """Loads a data file into a list of `InputBatch`s."""

        label_map = {label : i for i, label in enumerate(label_list)}

        features = []
        for (ex_index, example) in enumerate(examples):
            tokens_a = tokenizer.tokenize(example.text_a)

            tokens_b = None
            if example.text_b:
                tokens_b = tokenizer.tokenize(example.text_b)
                # Modifies `tokens_a` and `tokens_b` in place so that the total
                # length is less than the specified length.
                # Account for [CLS], [SEP], [SEP] with "- 3"
                self._truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
            else:
                # Account for [CLS] and [SEP] with "- 2"
                if len(tokens_a) > max_seq_length - 2:
                    tokens_a = tokens_a[:(max_seq_length - 2)]

            # The convention in BERT is:
            # (a) For sequence pairs:
            #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
            #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
            # (b) For single sequences:
            #  tokens:   [CLS] the dog is hairy . [SEP]
            #  type_ids: 0   0   0   0  0     0 0
            #
            # Where "type_ids" are used to indicate whether this is the first
            # sequence or the second sequence. The embedding vectors for `type=0` and
            # `type=1` were learned during pre-training and are added to the wordpiece
            # embedding vector (and position vector). This is not *strictly* necessary
            # since the [SEP] token unambigiously separates the sequences, but it makes
            # it easier for the model to learn the concept of sequences.
            #
            # For classification tasks, the first vector (corresponding to [CLS]) is
            # used as as the "sentence vector". Note that this only makes sense because
            # the entire model is fine-tuned.
            tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
            segment_ids = [0] * len(tokens)

            if tokens_b:
                tokens += tokens_b + ["[SEP]"]
                segment_ids += [1] * (len(tokens_b) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length
            #print("label_map >> ",label_map)
            label_id = label_map[example.label]
            if ex_index < 5:
                print("*** Example ***")
                print("guid: %s" % (example.guid))
                print("tokens: %s" % " ".join(
                        [str(x) for x in tokens]))
                print("input_ids: %s" % " ".join([str(x) for x in input_ids]))
                print("input_mask: %s" % " ".join([str(x) for x in input_mask]))
                print(
                        "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
                print("label: %s (id = %d)" % (example.label, label_id))

            features.append(
                    InputFeatures(input_ids=input_ids,
                                  input_mask=input_mask,
                                  segment_ids=segment_ids,
                                  label_id=label_id))
        return features


    def _truncate_seq_pair(self, tokens_a, tokens_b, max_length):
        """Truncates a sequence pair in place to the maximum length."""

        # This is a simple heuristic which will always truncate the longer sequence
        # one token at a time. This makes more sense than truncating an equal percent
        # of tokens from each, since if one sequence is very short then each token
        # that's truncated likely contains more information than a longer sequence.
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

    def accuracy(self, out, labels):
        outputs = np.argmax(out, axis=1)
        return np.sum(outputs == labels)

    def warmup_linear(self, x, warmup=0.002):
        if x < warmup:
            return x/warmup
        return 1.0 - x


In [4]:
class TrainClassifier():
    def __init__(self, data_dir, bert_model, task_name, do_train, do_eval, do_lower_case , output_dir,
                 labels, max_seq_length , train_batch_size , eval_batch_size ,learning_rate ,
                 num_train_epochs,  warmup_proportion, no_cuda, local_rank, seed,
                 gradient_accumulation_steps, fp16, loss_scale):
        """
        Args:
        data_dir : Directory containing data
        bert_model : Name of the pretrained model
        task_name : Name of the task
        output_dir : Directory to store model
        labels : List of target labels
        do_train : Boolean
        do_eval : Boolean
        max_seq_length : Maximum words in a sequence
        do_lower_case : Boolean
        train_batch_size : Training batch size
        eval_batch_size  :  Evaluation batch size
        learning_rate :  Learning rate (Initial)
        num_train_epochs : Number of training clycles
        warmup_proportion : Proportion of training to perform linear learning rate warmup 
        no_cuda : Boolean
        local_rank : local_rank for distributed training on gpus
        seed : Seed to get same results
        gradient_accumulation_steps : Number of updates steps to accumulate before performing a backward/update pass.
        fp16 : Boolean
        loss_scale : Loss scaling to improve fp16 numeric stability

        """
        self.data_dir = data_dir
        self.bert_model= bert_model
        self.task_name = task_name
        self.output_dir = output_dir
        self.labels = labels

        # other parameters
        self.do_train = do_train
        self.do_eval = do_eval
        self.max_seq_length = max_seq_length
        self.do_lower_case = do_lower_case
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size
        self.learning_rate = learning_rate
        self.num_train_epochs = num_train_epochs
        self.learning_rate =learning_rate 
        self.warmup_proportion = warmup_proportion
        self.no_cuda = no_cuda
        self.local_rank = local_rank 
        self.seed = seed
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.fp16 = fp16
        self.loss_scale = loss_scale
        

    
    def train(self, processor, num_labels, label_list, tokenizer, device, n_gpu):
        
        train_examples = None
        num_train_steps = None
        
        train_examples = processor.get_train_examples(self.data_dir)
        num_train_steps = int(
            len(train_examples) / self.train_batch_size / self.gradient_accumulation_steps * self.num_train_epochs)

        # Prepare model
        model = BertForSequenceClassification.from_pretrained(self.bert_model,
                  cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(self.local_rank),
                  num_labels = num_labels)
        
        if self.fp16:
            model.half()
        model.to(device)
        if self.local_rank != -1:
            try:
                from apex.parallel import DistributedDataParallel as DDP
            except ImportError:
                raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

            model = DDP(model)
        elif n_gpu > 1:
            model = torch.nn.DataParallel(model)

        # Prepare optimizer
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
        t_total = num_train_steps
        if self.local_rank != -1:
            t_total = t_total // torch.distributed.get_world_size()
        if self.fp16:
            try:
                from apex.optimizers import FP16_Optimizer
                from apex.optimizers import FusedAdam
            except ImportError:
                raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

            optimizer = FusedAdam(optimizer_grouped_parameters,
                                  lr=self.learning_rate,
                                  bias_correction=False,
                                  max_grad_norm=1.0)
            if self.loss_scale == 0:
                optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
            else:
                optimizer = FP16_Optimizer(optimizer, static_loss_scale=self.loss_scale)

        else:
            optimizer = BertAdam(optimizer_grouped_parameters,
                                 lr=self.learning_rate,
                                 warmup=self.warmup_proportion,
                                 t_total=t_total)

        global_step = 0
        if self.do_train:
            train_features = processor.convert_examples_to_features(
                train_examples, label_list, self.max_seq_length, tokenizer)
            print("***** Running training *****")
            print("  Num examples = %d", len(train_examples))
            print("  Batch size = %d", self.train_batch_size)
            print("  Num steps = %d", num_train_steps)
            all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
            all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
            all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
            all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
            train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
            if self.local_rank == -1:
                train_sampler = RandomSampler(train_data)
            else:
                train_sampler = DistributedSampler(train_data)
            train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.train_batch_size)

            model.train()
            for _ in trange(int(self.num_train_epochs), desc="Epoch"):
                tr_loss = 0
                nb_tr_examples, nb_tr_steps = 0, 0
                for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                    batch = tuple(t.to(device) for t in batch)
                    input_ids, input_mask, segment_ids, label_ids = batch
                    loss = model(input_ids, segment_ids, input_mask, label_ids)
                    if n_gpu > 1:
                        loss = loss.mean() # mean() to average on multi-gpu.
                    if self.gradient_accumulation_steps > 1:
                        loss = loss / self.gradient_accumulation_steps

                    if self.fp16:
                        optimizer.backward(loss)
                    else:
                        loss.backward()

                    tr_loss += loss.item()
                    nb_tr_examples += input_ids.size(0)
                    nb_tr_steps += 1
                    if (step + 1) % self.gradient_accumulation_steps == 0:
                        # modify learning rate with special warm up BERT uses
                        lr_this_step = self.learning_rate * processor.warmup_linear(global_step/t_total, self.warmup_proportion)
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr_this_step
                        optimizer.step()
                        optimizer.zero_grad()
                        global_step += 1
        model.to("cpu")
        # Save a trained model
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(self.output_dir, "pytorch_model.bin")
        torch.save(model_to_save.state_dict(), output_model_file)
        
        # Load a trained model that you have fine-tuned
        model_state_dict = torch.load(output_model_file)
        model = BertForSequenceClassification.from_pretrained(self.bert_model, state_dict= model_state_dict, num_labels =num_labels)
        model.to(device)

        if self.do_eval and (self.local_rank == -1 or torch.distributed.get_rank() == 0):
            eval_examples = processor.get_dev_examples(self.data_dir)
            eval_features = processor.convert_examples_to_features(
                eval_examples, label_list, self.max_seq_length, tokenizer)
            print("***** Running evaluation *****")
            print("  Num examples = %d", len(eval_examples))
            print("  Batch size = %d", self.eval_batch_size)
            all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
            all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
            all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
            all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
            print("all_label_ids >>> ", all_label_ids)
            eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
            # Run prediction for full data
            eval_sampler = SequentialSampler(eval_data)
            eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=self.eval_batch_size)

            model.eval()
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0
            for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
                input_ids = input_ids.to(device)
                input_mask = input_mask.to(device)
                segment_ids = segment_ids.to(device)
                label_ids = label_ids.to(device)
                
                with torch.no_grad():
                    tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
                    logits = model(input_ids, segment_ids, input_mask)

                logits = logits.detach().cpu().numpy()
                label_ids = label_ids.to('cpu').numpy()
                tmp_eval_accuracy = processor.accuracy(logits, label_ids)

                eval_loss += tmp_eval_loss.mean().item()
                eval_accuracy += tmp_eval_accuracy

                nb_eval_examples += input_ids.size(0)
                nb_eval_steps += 1

            eval_loss = eval_loss / nb_eval_steps
            eval_accuracy = eval_accuracy / nb_eval_examples

            result = {'eval_loss': eval_loss,
                      'eval_accuracy': eval_accuracy,
                      'global_step': global_step,
                      'loss': tr_loss/nb_tr_steps}

            output_eval_file = os.path.join(self.output_dir, "eval_results.txt")
            with open(output_eval_file, "w") as writer:
                print("***** Eval results *****")
                for key in sorted(result.keys()):
                    print("  %s = %s", key, str(result[key]))
                    writer.write("%s = %s\n" % (key, str(result[key])))
                
        
    def main(self):
        """ Initializes nesecarry parameters and runs the train function """
        
        if self.local_rank == -1 or self.no_cuda:
            device = torch.device("cuda" if torch.cuda.is_available() and not self.no_cuda else "cpu")
            n_gpu = torch.cuda.device_count()
        else:
            torch.cuda.set_device(self.local_rank)
            device = torch.device("cuda", self.local_rank)
            n_gpu = 1
            # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
            torch.distributed.init_process_group(backend='nccl')
        print("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
            device, n_gpu, bool(self.local_rank != -1), self.fp16))

        if self.gradient_accumulation_steps < 1:
            raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                                self.gradient_accumulation_steps))

        self.train_batch_size = int(self.train_batch_size / self.gradient_accumulation_steps)

        random.seed(self.seed)
        np.random.seed(self.seed)
        torch.manual_seed(self.seed)
        if n_gpu > 0:
            torch.cuda.manual_seed_all(self.seed)

        if not self.do_train and not self.do_eval:
            raise ValueError("At least one of `do_train` or `do_eval` must be True.")

        if os.path.exists(self.output_dir) and os.listdir(self.output_dir):
            raise ValueError("Output directory ({}) already exists and is not empty.".format(self.output_dir))
        os.makedirs(self.output_dir, exist_ok=True)

        task_name = self.task_name.lower()
        
        processors = { "toxic_comments": ToxicCommentsProcessor }

        num_labels_task = { "toxic_comments": len(self.labels) }
        
        if task_name not in processors:
            raise ValueError("Task not found: %s" % (task_name))

        processor = processors[task_name]()
        num_labels = num_labels_task[task_name]
        label_list = self.labels

        tokenizer = BertTokenizer.from_pretrained(self.bert_model, do_lower_case=self.do_lower_case)
        
        self.train(processor, num_labels, label_list, tokenizer, device, n_gpu)

In [ ]:
# run the classifier calling the necessary parameters from the config file
classifierObj = TrainClassifier(data_dir, bert_model, task_name, do_train, do_eval, do_lower_case , output_dir,
                 labels, max_seq_length , train_batch_size , eval_batch_size ,learning_rate ,
                 num_train_epochs,  warmup_proportion, no_cuda, local_rank, seed,
                 gradient_accumulation_steps, fp16, loss_scale)
classifierObj.main()

device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
*** Example ***
guid: train-0
tokens: [CLS] what is up pcs ? [SEP]
input_ids: 101 2054 2003 2039 27019 1029 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label: 0 (id = 0)
*** Example ***
guid: train-1
tokens: [CLS] how was california federal bank founded ? [SEP]
input_ids: 101 2129 2001 2662 2976 2924 2631 1029 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
seg

Iteration:   0%|          | 120/36735 [00:30<2:35:30,  3.92it/s]


Iteration:   1%|          | 240/36735 [01:01<2:36:34,  3.88it/s]


Iteration:   1%|          | 360/36735 [01:32<2:36:43,  3.87it/s]


Iteration:   1%|▏         | 480/36735 [02:03<2:37:14,  3.84it/s]


...

Iteration:  78%|███████▊  | 28666/36735 [2:05:38<35:17,  3.81it/s]


Iteration:  78%|███████▊  | 28783/36735 [2:06:09<35:34,  3.73it/s]


Iteration:  79%|███████▊  | 28900/36735 [2:06:40<34:03,  3.83it/s]


Iteration:  79%|███████▉  | 29017/36735 [2:07:10<33:44,  3.81it/s]


Iteration:  79%|███████▉  | 29081/36735 [2:07:27<33:10,  3.85it/s]